In [1]:
import requests
from bs4 import BeautifulSoup
from time import sleep # Задержка
import pandas as pd

### Первый фильм

In [2]:
url = 'https://www.kinopoisk.ru/lists/movies/top250/'

r = requests.get(url)
# r.text # html - страница

soup = BeautifulSoup(r.text, 'lxml') # Возвращать ответ в формате lxml
# soup

In [3]:
# Поиск элемента find() - для поиска по дереву, а get() для доступа к атрибутам, .text - очистить строку от тегов
# soup.find('div', class_='styles_root__ti07r')
link = 'https://www.kinopoisk.ru' + soup.find('div', class_='styles_root__ti07r').find('a', class_='base-movie-main-info_link__YwtP1').get('href')
print(link)
russian_name = soup.find('div', class_='styles_root__ti07r').find('a', class_='base-movie-main-info_link__YwtP1')\
.find('span', class_='styles_mainTitle__IFQyZ styles_activeMovieTittle__kJdJj').text
print(russian_name)

origin_name = soup.find('div', class_='styles_root__ti07r').find('a', class_='base-movie-main-info_link__YwtP1')\
.find('span', class_='desktop-list-main-info_secondaryTitle__ighTt').text
print(origin_name)

year_and_time = soup.find('div', class_='styles_root__ti07r').find('a', class_='base-movie-main-info_link__YwtP1')\
.find('span', class_='desktop-list-main-info_secondaryText__M_aus').text
year_and_time = year_and_time.split(',')
print(year_and_time[1].replace(' ', ''))
print(year_and_time[2].replace(' ', ''))

country_and_genre = soup.find('div', class_='styles_root__ti07r').find('a', class_='base-movie-main-info_link__YwtP1')\
.find('span', class_='desktop-list-main-info_truncatedText__IMQRP').text

country_and_genre = country_and_genre.split('•')

country = country_and_genre[0]
print(country)

genre = country_and_genre
genre_and_producer = country_and_genre[1].split('\xa0\xa0')
genre = genre_and_producer[0].replace(' ', '')
print(genre)


producer = genre_and_producer[1].replace('Режиссёр: ', '')
print(producer)

# В ролях одинаковый класс, поэтому функция find() не подходит, нужно использовать findAll() - возвращает список с результатами
role = soup.find('div', class_='styles_root__ti07r').find('a', class_='base-movie-main-info_link__YwtP1')\
.findAll('span', class_='desktop-list-main-info_truncatedText__IMQRP')[1].text

print(role)

score = soup.find('div', class_='styles_root__ti07r').find('span', class_='styles_kinopoiskValuePositive__vOb2E styles_kinopoiskValue__9qXjg styles_top250Type__mPloU').text
print(score)

https://www.kinopoisk.ru/film/435/
Зеленая миля
The Green Mile
1999
189 мин.
США 
драма
Фрэнк Дарабонт
В ролях: Том Хэнкс, Дэвид Морс
9.2


In [4]:
soup.find('div', class_='styles_root__ti07r').find('a', class_='base-movie-main-info_link__YwtP1')

<a class="base-movie-main-info_link__YwtP1" data-tid="d4e8d214" href="/film/435/"><div class="base-movie-main-info_mainInfo__ZL_u3" data-tid="eb1cb4b8"><span class="styles_mainTitle__IFQyZ styles_activeMovieTittle__kJdJj" data-tid="4502216a">Зеленая миля</span></div><div class="desktop-list-main-info_secondaryTitleSlot__mc0mI"><span class="desktop-list-main-info_secondaryTitle__ighTt">The Green Mile</span><span class="desktop-list-main-info_secondaryText__M_aus">, <!-- -->1999, 189 мин.</span></div><div class="desktop-list-main-info_additionalInfo__Hqzof" data-tid="31fba632"><span class="desktop-list-main-info_truncatedText__IMQRP">США • драма  Режиссёр: Фрэнк Дарабонт</span></div><div class="desktop-list-main-info_additionalInfo__Hqzof" data-tid="31fba632"><span class="desktop-list-main-info_truncatedText__IMQRP">В ролях: Том Хэнкс, Дэвид Морс</span></div></a>

## Для всех фильмов

In [7]:
data = []
for page in range(1, 6):
    print(page)
    url = f'https://www.kinopoisk.ru/lists/movies/top250/?page={page}'
    r = requests.get(url)
    # r.text # html - страница
    sleep(4)
    soup = BeautifulSoup(r.text, 'lxml') # Возвращать ответ в формате lxml


    films = soup.findAll('div', class_='styles_root__ti07r')
    # len(films) # Одна страница
    # print(films)

    for film in films:
        # Ссылка
        link = 'https://www.kinopoisk.ru' + film.find('a', class_='base-movie-main-info_link__YwtP1').get('href')
        # print(link)
        # Название на русском языке
        russian_name = film.find('a', class_='base-movie-main-info_link__YwtP1')\
            .find('span', class_='styles_mainTitle__IFQyZ styles_activeMovieTittle__kJdJj').text
        # print(russian_name)

        # Оригинальное название
        origin_name = film.find('a', class_='base-movie-main-info_link__YwtP1')\
            .find('span', class_='desktop-list-main-info_secondaryTitle__ighTt')
        # print(origin_name)
        if(origin_name != None):
            origin_name = origin_name.text
            # print(origin_name)
        else: 
            print()
            # print(origin_name)

        # Год и продолжительность
        year_and_time = film.find('a', class_='base-movie-main-info_link__YwtP1')\
            .find('span', class_='desktop-list-main-info_secondaryText__M_aus').text
        # Проверка содержимого # print(year_and_time) 

        # Если < 16, то
        if(len(year_and_time) < 16):
            year_and_time = year_and_time.split(',')
            year = year_and_time[0].replace(' ', '')
            time = year_and_time[1].replace(' ', '').replace('\xa0', ' ')
            # print(year)
            # print(time)
        else:
            year_and_time = year_and_time.split(',')
            year = year_and_time[1].replace(' ', '')
            time = year_and_time[2].replace(' ', '').replace('\xa0', ' ')
            # print(year)
            # print(time)

        country_and_genre = film.find('a', class_='base-movie-main-info_link__YwtP1')\
            .find('span', class_='desktop-list-main-info_truncatedText__IMQRP').text
        # Страна
        country_and_genre = country_and_genre.split('•')
        # # 222print(country_and_genre)
        country = country_and_genre[0]

        # print(country)
        # Жанр
        genre_and_producer = country_and_genre[1].split('\xa0\xa0')
        genre = genre_and_producer[0].replace(' ', '')
        # print(genre)
        
        # Режиссёр 
        producer = genre_and_producer[1].replace('Режиссёр: ', '')
        # print(producer)

        # В ролях
        role = film.find('a', class_='base-movie-main-info_link__YwtP1')\
            .findAll('span', class_='desktop-list-main-info_truncatedText__IMQRP')[1].text
        # print(role)

        # Оценка кинопоиска
        # try except - если вдруг оценка есть не у всех фильмов
        try:
            score = film.find('span', class_='styles_kinopoiskValuePositive__vOb2E styles_kinopoiskValue__9qXjg styles_top250Type__mPloU').text
        except:
            score = '-'
        # print(score)

        data.append([link, russian_name, origin_name, year, time, country, genre, producer])

# data

1
2
3









4
5


In [8]:
print(len(data))
header = ['link', 'russian_name', 'origin_name', 'year', 'time', 'country', 'genre', 'producer']
df = pd.DataFrame(data, columns=header)
df

50


,link,russian_name,origin_name,year,time,country,genre,producer
0,https://www.kinopoisk.ru/film/470191/,Невероятный мир глазами Энцо,The Art of Racing in the Rain,2019,109 мин.,США,драма,Саймон Кёртис
1,https://www.kinopoisk.ru/film/89515/,Гарри Поттер и Принц-полукровка,Harry Potter and the Half-Blood Prince,2009,153 мин.,Великобритания,фэнтези,Дэвид Йейтс
2,https://www.kinopoisk.ru/film/2656/,Пятый элемент,The Fifth Element,1997,126 мин.,Франция,фантастика,Люк Бессон
3,https://www.kinopoisk.ru/film/7724/,Летят журавли,None,1957,95 мин.,СССР,военный,Михаил Калатозов
4,https://www.kinopoisk.ru/film/8125/,Красотка,Pretty Woman,1990,115 мин.,США,мелодрама,Гэрри Маршалл
5,https://www.kinopoisk.ru/film/8221/,Мой сосед Тоторо,Tonari no Totoro,,1988,Япония,аниме,Хаяо Миядзаки
6,https://www.kinopoisk.ru/film/843649/,Мстители: Война бесконечности,Avengers: Infinity War,2018,149 мин.,США,фантастика,Энтони Руссо
7,https://www.kinopoisk.ru/film/346/,12 разгневанных мужчин,12 Angry Men,,1956,США,драма,Сидни Люмет
8,https://www.kinopoisk.ru/film/848894/,Вторая жизнь Уве,En man som heter Ove,2015,116 мин.,Швеция,драма,Ханнес Холм
9,https://www.kinopoisk.ru/film/16117/,Лис и пёс,The Fox and the Hound,,1981,США,мультфильм,Тед Берман


In [9]:
# Старан с наибольшим кол-во фильмов в топ 250 кинопоиска
df.country.value_counts()

США                29
СССР                6
Франция             2
Германия            2
Россия              2
Великобритания      2
Новая Зеландия      2
Италия              1
Швеция              1
Беларусь            1
Япония              1
Австралия           1
Name: country, dtype: int64

In [10]:
# Самый популярный жанр в топ 250 кинопоиска
df.genre.value_counts()

мультфильм     15
драма          12
фэнтези         5
фантастика      4
военный         3
мелодрама       2
боевик          2
комедия         1
приключения     1
аниме           1
триллер         1
криминал        1
биография       1
детектив        1
Name: genre, dtype: int64

In [11]:
# Самвй популрный жанр по странам
df.pivot_table(index=['country', 'genre'], values='russian_name', aggfunc='count')
# df.groupby(['country', 'genre'])[['link']].count()

russian_name
country         genre                    
Австралия       драма                   1
Беларусь        военный                 1
Великобритания  фэнтези                 2
Германия        боевик                  1
                фантастика              1
Италия          приключения             1
Новая Зеландия  фэнтези                 2
Россия          боевик                  1
                драма                   1
СССР            военный                 2
                драма                   2
                комедия                 1
                криминал                1
США             биография               1
                детектив                1
                драма                   7
                мелодрама               2
                мультфильм             14
                триллер                 1
                фантастика              2
                фэнтези                 1
Франция         мультфильм              1
                фантастика              1
Швеция          драма                   1
Япония          аниме                   1